### Ingestion del archivo language_role.json


In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
languages_roles_schema = StructType(fields=[
     StructField("roleId", IntegerType(), True),
     StructField("languageRole", StringType(), True),
])

In [0]:
#Agregamos v_file_date, ya que ahora en nuestro Data Lake, tenemos carpetas con la fecha
#Seccion carga incormental, para esta carga por ser pocos datos, sera una carga completa
language_role_df = spark.read \
                   .schema(languages_roles_schema) \
                   .option("multiLine", True) \
                   .json(f"{bronze_folder_path}/{v_file_date}/language_role.json")

### Paso 2 - Renombrar las columnas y añadir columnas nuevas
1. "roleId" renombrar a "role_id"
2. "languageRole" renombrar a "language_role"
3. Agregar las columnas de "ingestion_date" y "envoronment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
language_role_final_df = add_ingestion_date(language_role_df) \
                          .withColumn("environment", lit(v_environment)) \
                          .withColumn("file_date", lit(v_file_date)) \
                          .withColumnsRenamed(
                             {"roleId": "role_id", 
                             "languageRole": "language_role"
                             }) 
                         

### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
# language_role_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/languages_roles")

# Crear una tabla en base al data frame language_role_final_df
# Creara la tabla languages_roles en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver
#language_role_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages_roles")

# Modulo Delta Lake
language_role_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages_roles")

In [0]:
%sql
SELECT * FROM movie_silver.languages_roles

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/languages_roles"))

In [0]:
dbutils.notebook.exit("Success")